In [1]:
from pyiron_workflow import Workflow

In [2]:
@Workflow.wrap.as_function_node
def HelloWorld(greeting="Hello", subject="World"):
    hello = f"{greeting} {subject}"
    return hello

In [3]:
hello_node = HelloWorld()  # Instantiate a node instance
hello_node(greeting="Salutations")  # Use it just like a function

'Salutations World'

In [5]:
print(hello_node.inputs.labels)

hello_node.outputs.hello.value

['greeting', 'subject']


'Salutations World'